In [1]:
import nltk
import pandas as pd
import re
import numpy as np
from pprint import pprint

nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


addedWords = 'created_utc','selftext','title','have', 'be', 'html', 'http', 'https', 'just', 'know', 'not'
stop_words.extend(addedWords)



df = pd.read_json('/home/inez/Documents/jupyternotebookfiles/jsonDumpSubmissionBigrams2.json')
#10,010 records

df.head(1)



[nltk_data] Downloading package stopwords to /home/inez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/inez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,"{'created_utc': 1514765030, 'selftext': '', 't...","{'created_utc': 1514765175, 'title': 'Empty Ju...","{'created_utc': 1514765208, 'selftext': 'Hey g...","{'created_utc': 1514765358, 'selftext': 'My co...","{'created_utc': 1514765775, 'selftext': '[remo...","{'created_utc': 1514766068, 'selftext': 'I jus...","{'created_utc': 1514766327, 'selftext': 'All, ...","{'created_utc': 1514767057, 'selftext': 'Just ...","{'created_utc': 1514767313, 'selftext': 'Hello...","{'created_utc': 1514767535, 'selftext': '[remo...",...,"{'created_utc': 1515326842, 'title': '[WTS] Em...","{'created_utc': 1515327059, 'selftext': 'Hey g...","{'created_utc': 1515327448, 'selftext': '[remo...","{'created_utc': 1515328717, 'selftext': 'Hey g...","{'created_utc': 1515328896, 'selftext': 'I rea...","{'created_utc': 1515330004, 'selftext': '', 't...","{'created_utc': 1515333905, 'selftext': '', 't...","{'created_utc': 1515335453, 'selftext': '▓▓▓▓▓...","{'created_utc': 1515335616, 'selftext': 'Right...","{'created_utc': 1515336167, 'selftext': 'Hey a..."


In [2]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#convert to list
titledata = df.values.T.tolist()
type(titledata)
print(titledata[0])
pprint(titledata[:1])
testdict = titledata[0]

[{'created_utc': 1514765030, 'selftext': '', 'title': 'Not juul related, but whats better, Suorin Air vs Drop, whats better?'}, {'created_utc': 1515336462, 'selftext': 'I was sitting the other day in the morning, trying to wake up with my coffee (as I like it, I have a brewing machine, so it is a good coffee) and with my vape machine. Took a sip, then vaped my DIY Unicorn Milk (never had the original, not avaible in my country) and I just loved it. It was like regular, strong black coffee with no sugar nor milk, and then after vaping, the taste left on my tounge was a great strawberry latte. And I was curious, do any of you guys have any combo with food/drink and vape? Good beer with some juices goes well too, but nothing as remarkable as this coffee/unicorn milk. Let me know!', 'title': 'What is your fav food/drink and vape combo?'}, {'created_utc': 1515844841, 'selftext': "So I'm using Smok tfv8 coils on my baby beast atty, and I'm not sure if this is normal, but after 4 or 5 days of

In [4]:
# Flatten titledata
titledata = [data for entry in titledata for data in entry]
for s in titledata:
    if s:
        s['title'] = s['title'].replace('\n', '') if s.get('title') else None
        s['selftext'] = s['selftext'].replace('\n', '') if s.get('selftext') else None


type(titledata)

list

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(titledata))

print(data_words[0])
type(data_words)

['created_utc', 'selftext', 'none', 'title', 'not', 'juul', 'related', 'but', 'whats', 'better', 'suorin', 'air', 'vs', 'drop', 'whats', 'better']


list

In [6]:
result=[]
#iterate over each inner i list in data_words
for i in data_words:
    new=[]
#iterate over word inside the inner i list
    for word in i:
#if word not in stop_words, then i remove it
        if word not in stop_words:
            new.append(word)
            
            
    result.append(new)
    


In [7]:
type(result)

list

In [8]:
print(result[:2])

[['none', 'juul', 'related', 'whats', 'better', 'suorin', 'air', 'vs', 'drop', 'whats', 'better'], ['sitting', 'day', 'morning', 'trying', 'wake', 'coffee', 'like', 'brewing', 'machine', 'good', 'coffee', 'vape', 'machine', 'took', 'sip', 'vaped', 'diy', 'unicorn', 'milk', 'never', 'original', 'avaible', 'country', 'loved', 'like', 'regular', 'strong', 'black', 'coffee', 'sugar', 'milk', 'vaping', 'taste', 'left', 'tounge', 'great', 'strawberry', 'latte', 'curious', 'guys', 'combo', 'food', 'drink', 'vape', 'good', 'beer', 'juices', 'goes', 'well', 'nothing', 'remarkable', 'coffee', 'unicorn', 'milk', 'let', 'fav', 'food', 'drink', 'vape', 'combo']]


In [9]:
count=0
for innerlist in result:
    if word in innerlist in stop_words:
        count+=1
        print(word)
print("stop words found are ", count)


stop words found are  0


In [10]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')

stemmedList=[]
for innerlist in result:
#test = result[0]
    new = []
    for t in innerlist:
    #print (t + '-->' + stemmer.stem(t))
        new.append(stemmer.stem(t))
    stemmedList.append(new)
#print(test)
print(stemmedList[:2])

[['none', 'juul', 'relat', 'what', 'better', 'suorin', 'air', 'vs', 'drop', 'what', 'better'], ['sit', 'day', 'morn', 'tri', 'wake', 'coffe', 'like', 'brew', 'machin', 'good', 'coffe', 'vape', 'machin', 'took', 'sip', 'vape', 'diy', 'unicorn', 'milk', 'never', 'origin', 'avaibl', 'countri', 'love', 'like', 'regular', 'strong', 'black', 'coffe', 'sugar', 'milk', 'vape', 'tast', 'left', 'toung', 'great', 'strawberri', 'latt', 'curious', 'guy', 'combo', 'food', 'drink', 'vape', 'good', 'beer', 'juic', 'goe', 'well', 'noth', 'remark', 'coffe', 'unicorn', 'milk', 'let', 'fav', 'food', 'drink', 'vape', 'combo']]


In [11]:
#Create Dictionary
id2word = corpora.Dictionary(stemmedList)

# Create Corpus
texts = stemmedList

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:5])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2)], [(9, 1), (10, 1), (11, 1), (12, 1), (13, 4), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 2), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 2), (33, 3), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 5), (51, 1), (52, 1)], [(17, 2), (50, 1), (53, 1), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 5), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 2), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 3), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 2), (88, 1)], [(3, 3), (48, 1), (52, 1), (72, 1), (73, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 2), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), 

In [12]:
#tokenization with gensim preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


In [13]:
data_words = list(sent_to_words(stemmedList))
print(len(stemmedList))
print(stemmedList[0])

11000
['none', 'juul', 'relat', 'what', 'better', 'suorin', 'air', 'vs', 'drop', 'what', 'better']


In [14]:
type(stemmedList)

list

In [15]:
#Gensim creates a unique id for each word in the document. 
#The produced corpus shown 
#above is a mapping of (word_id, word_frequency).
#For example, (0, 1) above implies, word id 0 occurs once in the first document.
#Likewise, word id 1 occurs twice and so on.
id2word[531]

'thing'

In [16]:
id2word[204]

'found'

In [17]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

[[('air', 1),
  ('better', 2),
  ('drop', 1),
  ('juul', 1),
  ('none', 1),
  ('relat', 1),
  ('suorin', 1),
  ('vs', 1),
  ('what', 2)],
 [('avaibl', 1),
  ('beer', 1),
  ('black', 1),
  ('brew', 1),
  ('coffe', 4),
  ('combo', 2),
  ('countri', 1),
  ('curious', 1),
  ('day', 1),
  ('diy', 1),
  ('drink', 2),
  ('fav', 1),
  ('food', 2),
  ('goe', 1),
  ('good', 2),
  ('great', 1),
  ('guy', 1),
  ('juic', 1),
  ('latt', 1),
  ('left', 1),
  ('let', 1),
  ('like', 2),
  ('love', 1),
  ('machin', 2),
  ('milk', 3),
  ('morn', 1),
  ('never', 1),
  ('noth', 1),
  ('origin', 1),
  ('regular', 1),
  ('remark', 1),
  ('sip', 1),
  ('sit', 1),
  ('strawberri', 1),
  ('strong', 1),
  ('sugar', 1),
  ('tast', 1),
  ('took', 1),
  ('toung', 1),
  ('tri', 1),
  ('unicorn', 2),
  ('vape', 5),
  ('wake', 1),
  ('well', 1)]]

In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [19]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.058*"gb" + 0.045*"fun" + 0.042*"lead" + 0.040*"compat" + 0.023*"rise" + '
  '0.022*"travel" + 0.021*"ny" + 0.020*"beer" + 0.015*"februari" + '
  '0.011*"pussi"'),
 (1,
  '0.151*"none" + 0.147*"juul" + 0.071*"pod" + 0.035*"remov" + 0.026*"pen" + '
  '0.025*"hit" + 0.019*"addict" + 0.012*"wts" + 0.012*"squonk" + 0.012*"ship"'),
 (2,
  '0.050*"vape" + 0.019*"use" + 0.015*"like" + 0.014*"get" + 0.012*"new" + '
  '0.011*"look" + 0.010*"coil" + 0.010*"juic" + 0.010*"good" + 0.008*"tri"'),
 (3,
  '0.105*"com" + 0.069*"www" + 0.031*"reddit" + 0.027*"comment" + '
  '0.019*"thread" + 0.018*"question" + 0.017*"post" + 0.016*"vape" + '
  '0.013*"youtub" + 0.012*"point"'),
 (4,
  '0.040*"tr" + 0.040*"com" + 0.040*"lti" + 0.035*"imgur" + 0.035*"png" + '
  '0.033*"origin" + 0.023*"aegi" + 0.017*"misc" + 0.016*"anvil" + '
  '0.015*"dynam"'),
 (5,
  '0.031*"mod" + 0.023*"com" + 0.020*"batteri" + 0.016*"vapor" + 0.015*"amp" + '
  '0.014*"tank" + 0.014*"kit" + 0.013*"imgur" + 0.012*"coil" + 0.0

In [23]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.



Perplexity:  -7.6935236591648986


In [24]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=stemmedList, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.46769119948363347


In [25]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/inez/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.273895 -0.216245       1        1  29.331020
2      0.308010 -0.154271       2        1  28.288877
4      0.102030  0.346904       3        1  18.993116
5      0.171632  0.085500       4        1  11.293726
8     -0.106252  0.105655       5        1   3.557200
3      0.025000  0.122259       6        1   3.067681
1     -0.113617  0.009553       7        1   3.033390
7     -0.217624 -0.095245       8        1   1.322195
9     -0.225151 -0.100120       9        1   0.739658
0     -0.217921 -0.103990      10        1   0.373143, topic_info=      Category          Freq        Term         Total  loglift  logprob
686    Default  13051.000000         com  13051.000000  30.0000  30.0000
50     Default  15106.000000        vape  15106.000000  29.0000  29.0000
4      Default   4483.000000        none   4483.000000  28.0000  28.0000
3      Default   4364.000000        juul   4364.000000  27.0000  27.0000
244    Default   5552.000000     comment   5552.000000  26.0000  26.0000
1989   Default   7517.000000          tr   7517.000000  25.0000  25.0000
1854   Default   7362.000000         lti   7362.000000  24.0000  24.0000
1914   Default   6464.000000         png   6464.000000  23.0000  23.0000
5922   Default   3132.000000    xymarket   3132.000000  22.0000  22.0000
673    Default   3307.000000         www   3307.000000  21.0000  21.0000
1147   Default   8243.000000       imgur   8243.000000  20.0000  20.0000
211    Default   2936.000000         pod   2936.000000  19.0000  19.0000
1492   Default   2885.000000      paypal   2885.000000  18.0000  18.0000
37     Default   6530.000000      origin   6530.000000  17.0000  17.0000
74     Default   4267.000000         mod   4267.000000  16.0000  16.0000
1686   Default   4350.000000        aegi   4350.000000  15.0000  15.0000
166    Default   4507.000000       smoke   4507.000000  14.0000  14.0000
68     Default   2073.000000         hit   2073.000000  13.0000  13.0000
1488   Default   3174.000000        misc   3174.000000  12.0000  12.0000
232    Default   3744.000000         new   3744.000000  11.0000  11.0000
1694   Default   3028.000000       anvil   3028.000000  10.0000  10.0000
1068   Default   2642.000000     batteri   2642.000000   9.0000   9.0000
1771   Default   2723.000000       dynam   2723.000000   8.0000   8.0000
505    Default   1476.000000      reddit   1476.000000   7.0000   7.0000
250    Default   1792.000000    question   1792.000000   6.0000   6.0000
160    Default   4440.000000         one   4440.000000   5.0000   5.0000
136    Default    894.000000          mg    894.000000   4.0000   4.0000
59     Default   4258.000000        coil   4258.000000   3.0000   3.0000
1757   Default   2473.000000      cyclon   2473.000000   2.0000   2.0000
30     Default   7233.000000        like   7233.000000   1.0000   1.0000
...        ...           ...         ...           ...      ...      ...
807    Topic10    163.642197         fun    164.565918   5.5853  -3.1064
802    Topic10    155.122223        lead    156.045822   5.5850  -3.1599
801    Topic10    146.669678      compat    147.593597   5.5847  -3.2159
294    Topic10     83.608948        rise     84.532867   5.5800  -3.7779
2166   Topic10     80.509743      travel     81.433655   5.5796  -3.8157
7842   Topic10     74.947136          ny     75.871056   5.5787  -3.8873
10     Topic10     71.999359        beer     72.922951   5.5782  -3.9274
2152   Topic10     53.027946    februari     53.951614   5.5737  -4.2333
7158   Topic10     39.736977       pussi     40.661350   5.5680  -4.5218
806    Topic10     33.242104         feb     34.165909   5.5636  -4.7003
3591   Topic10     32.745766      sexual     33.669571   5.5631  -4.7153
5555   Topic10     31.450945        poli     32.375648   5.5620  -4.7557
10848  Topic10     24.804838      narrat     25.728565   5.5544  -4.9931
3277   Topic10     21.86